In [1]:
import pandas as pd
import numpy as np
import pulp as pl
import warnings
warnings.filterwarnings("ignore")

### Leitura dos Dados

In [2]:
# df = pd.read_excel(
#     './data/portfolio_20250526.xlsx',
#     sheet_name='Portfolio ',
#     header=0,
#     skiprows=4,   # Pula as primeiras 4 linhas
#     nrows=68,     # Lê 68 linhas a partir da linha 5 (até a linha 72)
#     usecols='B:N' # Seleciona as colunas de B até N
# )

df_port = pd.read_csv('./data/portfolio_20250526_tratado.csv', sep=';', index_col=0)

df_port = df_port.dropna(how='all')

In [3]:
df_port.head()

,Geo.,Classe,Subclasses,Setor,Ativo,Ticker,Qnt.,Cotação,Cotação USD,Total,% Atual,% Ideal - Ref.,Variação
0,BR,RF,Juros Pós,-,Tesouro SELIC,SELIC,12649.370,1.0,NaN,12649.370,0.063830,0.124192,-0.060362
1,BR,RF,Juros Pós,-,Fundo DI Simples,FDI,0.000,1.0,NaN,0.000,0.000000,0.053531,-0.053531
2,BR,RF,Juros Pós,-,Fundo Renda Fixa High Grade Crédito Privado,FRFH,0.000,1.0,NaN,0.000,0.000000,0.032119,-0.032119
3,BR,RF,Juros Pós,-,Letra de Crédito (LCI/LCA),LC,6860.620,1.0,NaN,6860.620,0.034619,0.032119,0.002501
4,BR,RF,Juros Pós,-,CDB High Yield Pós-Fixado,CDB,34438.545,1.0,NaN,34438.545,0.173781,0.042825,0.130956


In [4]:
VALOR_CARTEIRA = df_port['Total'].sum()
VALOR_APORTE = 7250

### Estratégia 1: Rebalanceamento Simples por Déficit e redistribuição de sobras

In [5]:
def aporte_inicial(df, valor_carteira, valor_aporte):

    df = df.copy()

    df['Valor Ideal'] = df['% Ideal - Ref.']*(valor_carteira+valor_aporte)
    df['deficit'] = (df['Valor Ideal'] - df['Total']).clip(lower=0)

    df_grp = df.groupby(['Geo.', 'Classe', 'Subclasses', 'Ativo', 'Ticker'], as_index=False).agg(deficit = ('deficit', 'sum'))
    total_deficit = df_grp['deficit'].sum()
    df_grp['aporte'] = (df_grp['deficit']/total_deficit)*valor_aporte

    # Cruza para pegar Cotação e quantidade
    df_grp = pd.merge(
        df_grp,
        df[['Geo.', 'Classe', 'Subclasses', 'Ativo', 'Ticker', 'Qnt.','Cotação']],
        how='left',
        on=['Geo.', 'Classe', 'Subclasses', 'Ativo', 'Ticker']
    )

    # Calculo da quantidade necessaria, segundo valor do aporte
    # Para Renda Fixa de cotação = 1, calcular normalmente
    # Para Renda Variável com cotação maior que 1, calcular quantidade inteira possível

    df_grp['Qtd_nec'] = 0.0
    df_grp['Custo_real'] = 0.0

    df_grp.loc[(df_grp['Classe']=='RF') & (df_grp['Ticker']!='IMAB11'), 'Qtd_nec'] = df_grp['aporte'] / df_grp['Cotação']
    df_grp.loc[(df_grp['Classe']=='RF') & (df_grp['Ticker']!='IMAB11'), 'Custo_real'] = df_grp['aporte']

    df_grp.loc[(df_grp['Classe']!='RF') | (df_grp['Ticker']=='IMAB11'), 'Qtd_nec'] = np.floor(df_grp['aporte'] / df_grp['Cotação'])
    df_grp.loc[(df_grp['Classe']!='RF') | (df_grp['Ticker']=='IMAB11'), 'Custo_real'] = df_grp['Qtd_nec']*df_grp['Cotação']

    return df_grp

def calcula_sobra(df, valor_aporte):

    custo_total = df['Custo_real'].sum()
    vlr_sobra = valor_aporte - custo_total

    return np.max([0,vlr_sobra])

def menor_preco_viavel(df):

    elegiveis = df[(df['Classe'] != 'RF') & (df['deficit'] > 0) & (df['Cotação'] > 0)]

    return elegiveis['Cotação'].min() if not elegiveis.empty else np.inf

# def viabiliza_sobra(df, vlr_sobra):

#     menor_preco = df[(df['deficit']>0) & (df['Classe']=='RV')]['Cotação'].min()

#     return True if menor_preco <= vlr_sobra else False

def redistribui_sobra(df, vlr_sobra):

    df = df.copy()

    while vlr_sobra >= menor_preco_viavel(df) - 1e-6:

        alvo = df[(df['Classe'] != 'RF') & (df['deficit'] > 0)].sort_values('deficit', ascending=False).iloc[0]
        preco = alvo['Cotação']
        ativo = alvo['Ativo']
        deficit = alvo['deficit']
        if vlr_sobra < preco:
            break

        df.loc[df['Ativo']==ativo, 'Qtd_nec'] +=1
        df.loc[df['Ativo']==ativo, 'Custo_real'] +=preco

        vlr_sobra -= preco

        df.loc[df['Ativo']==ativo, 'deficit'] = max(0,deficit-preco)

    return df, vlr_sobra

def otimiza_aporte(df, valor_aporte):
        
    # valor_carteira = df['Total'].sum()

    base = aporte_inicial(df, valor_carteira=VALOR_CARTEIRA, valor_aporte=VALOR_APORTE)
    if base.empty:
        return base, valor_aporte

    vlr_sobra = calcula_sobra(base, valor_aporte=VALOR_APORTE)

    for _ in range(30):
        menor = menor_preco_viavel(base)
        if vlr_sobra < menor - 1e-6:
            break
        base, vlr_sobra = redistribui_sobra(base, vlr_sobra)

    return base, vlr_sobra

def exibir_resultado_formatado(df_resultado, sobra, valor_aporte):
    """Exibe resultado de forma padronizada e formatada."""

    resultado = df_resultado[df_resultado['Custo_real'] > 0].copy()
    resultado = resultado.sort_values('Custo_real', ascending=False).reset_index(drop=True)
    resultado = resultado[['Geo.', 'Classe', 'Subclasses', 'Ativo', 'Ticker', 
                          'Cotação', 'Qtd_nec', 'Custo_real']]
    
    custo_total = resultado['Custo_real'].sum()


    print(f"\nRESULTADO - REBALANCEAMENTO")
    print("="*60)
    display(resultado)
    
    print('\n' + '='*60)
    print('RESUMO FINANCEIRO:')
    print('='*60)
    print(f'Orçamento Total:    R$ {valor_aporte:,.2f}')
    print(f'Custo Total:        R$ {custo_total:,.2f}')
    print(f'Sobra:              R$ {sobra:,.2f}')
    print(f'Utilização:         {(custo_total/valor_aporte)*100:.1f}%')
    
    if sobra > 0:
        print(f'\nSobra de R$ {sobra:,.2f} vai para SELIC')

In [6]:
df_aporte, sobra_final = otimiza_aporte(df_port, VALOR_APORTE)
exibir_resultado_formatado(df_aporte, sobra_final, valor_aporte=VALOR_APORTE)


RESULTADO - REBALANCEAMENTO


,Geo.,Classe,Subclasses,Ativo,Ticker,Cotação,Qtd_nec,Custo_real
0,BR,RV,Criptoativos,Hashdex Crypto Index,HASH11,81.59,18.000000,1468.620000
1,BR,RF,Juros Pós,Tesouro SELIC,SELIC,1.00,802.045185,802.045185
2,BR,RF,Juros Pós,Fundo DI Simples,FDI,1.00,685.690121,685.690121
3,BR,RF,Inflação,Tesouro IPCA 2040,IPCA,1.00,411.414073,411.414073
4,BR,RF,Juros Pós,Fundo Renda Fixa High Grade Crédito Privado,FRFH,1.00,411.414073,411.414073
5,BR,RV,FIIs,Sparta Infra,JURO11,99.58,4.000000,398.320000
6,BR,RF,Inflação,CDB Inflação,CDBI,1.00,274.276048,274.276048
7,BR,RF,Inflação,Tesouro IPCA 2050,IPCA,1.00,260.562246,260.562246
8,BR,RF,Inflação,Tesouro IPCA 2029,IPCA,1.00,170.495325,170.495325
9,BR,RV,Ações,Banco do Brasil,BBAS3,24.67,6.000000,148.020000



RESUMO FINANCEIRO:
Orçamento Total:    R$ 7,250.00
Custo Total:        R$ 7,212.66
Sobra:              R$ 37.34
Utilização:         99.5%

Sobra de R$ 37.34 vai para SELIC


### Estratégia 2: Pesquisa Operacional
- Restrição: Valor aportado
- Função otimização: redução dos gaps residuais

Leitura recomendada: https://www.kaggle.com/code/farshadjafari97/operations-research-with-python

In [7]:
# def preparar_base(df, valor_carteira, valor_aporte):
#     df = df.copy()
#     df['Valor Ideal'] = df['% Ideal - Ref.'] * (valor_carteira + valor_aporte)
#     df['deficit'] = (df['Valor Ideal'] - df['Total']).clip(lower=0)
#     df['Qtd_nec'] = 0.0
#     df['Custo_real'] = 0.0
#     return df

def criar_variaveis_lp(df):
    '''
    Cria as variáveis de decisão para RF, RV e gaps.
    '''
    df = df.copy()

    qt_rf = {} # Dicionario para armazenar as variaveis de RF (continua) {id: 'nome_var_ativo'}
    qt_rv = {} # Dicionario para armazenar as variaveis de RV (inteiro) {id: 'nome_var_ativo'}
    gap = {} # Gaps residuais, ou seja, o que sobrou apos a compra (deficit - aporte)

    for idx, row in df.iterrows():
        ativo_i = row['Ativo'].replace(' ','_')
        preco_i = row['Cotação']
        deficit_i = row['deficit']

        if row['Classe']=='RF' and row['Ticker'] != 'IMAB11':
            qt_rf[idx] = pl.LpVariable(f'RF_{ativo_i}', lowBound=0) # Atribui variaveis RF
        else:
            qt_rv[idx] = pl.LpVariable(f'RV_{ativo_i}', lowBound=0, cat='Integer') # Atribui variaveis RV
        
        gap[idx] = pl.LpVariable(f'GAP_{ativo_i}', lowBound=0)

    return qt_rf, qt_rv, gap

def adicionar_restricoes_lp(prob, df, qt_rf, qt_rv, gap, valor_aporte):
    '''
    Adiciona todas as restrições ao problema de otimização.
    '''
    df = df.copy()
    # Calculo de gastos da cotação x qtd (xi)
    gasto_rf = pl.lpSum(df.loc[i,'Cotação']*qt_rf[i] for i in qt_rf)
    gasto_rv = pl.lpSum(df.loc[i,'Cotação']*qt_rv[i] for i in qt_rv)

    # Adiciona Restrição
    prob += (gasto_rf + gasto_rv) <= valor_aporte

    for idx, row in df.iterrows():
        preco_i = row['Cotação']
        deficit_i = row['deficit']

        if idx in qt_rf:
            compra_valor = preco_i*qt_rf[idx]
        else:
            compra_valor = preco_i*qt_rv[idx]

        # Restrição do gap
        prob += gap[idx] >= deficit_i - compra_valor
    
def definir_objetivo_lp(prob, gap):
    '''
    Define a função objetivo: minimizar soma dos gaps.
    '''
    prob += pl.lpSum(gap.values())

def extrair_resultados_lp(df, qt_rf, qt_rv, valor_aporte, show=True):
    '''
    Extrai os resultados da otimização e calcula custo total.
    '''
    df = df.copy()

    resultados = []
    custo_total = 0

    # Processar resultados RF
    for idx in qt_rf:
        qtd_comprada = qt_rf[idx].varValue
        if qtd_comprada > 0:
            resultado = {
                'Geo.': df.loc[idx, 'Geo.'],
                'Ticker': df.loc[idx, 'Ticker'],
                'Ativo': df.loc[idx, 'Ativo'],
                'Classe': df.loc[idx, 'Classe'],
                'Sublasses': df.loc[idx, 'Subclasses'],
                'Cotação': df.loc[idx, 'Cotação'],
                'Qtd_Comprada': qtd_comprada,
                'Valor_Compra': qtd_comprada * df.loc[idx, 'Cotação']
            }
            resultados.append(resultado)
            custo_total += resultado['Valor_Compra']
    
    # Processar resultados RV
    for idx in qt_rv:
        qtd_comprada = qt_rv[idx].varValue
        if qtd_comprada > 0:
            resultado = {
                'Geo.': df.loc[idx, 'Geo.'],
                'Ticker': df.loc[idx, 'Ticker'],
                'Ativo': df.loc[idx, 'Ativo'],
                'Classe': df.loc[idx, 'Classe'],
                'Sublasses': df.loc[idx, 'Subclasses'],
                'Cotação': df.loc[idx, 'Cotação'],
                'Qtd_Comprada': int(qtd_comprada),  # Inteiro para RV
                'Valor_Compra': qtd_comprada * df.loc[idx, 'Cotação']
            }
            
            resultados.append(resultado)
            custo_total += resultado['Valor_Compra']

    sobra = valor_aporte - custo_total

    if resultados:
        df_resultado = pd.DataFrame(resultados)
        df_resultado = df_resultado.sort_values('Valor_Compra', ascending=False)

        if show:
            print(f"\nRESULTADO - PESQUISA OPERACIONAL")
            print("="*60)
            display(df_resultado)
            print('\n' + '=' * 60)
            print('RESUMO FINANCEIRO:')
            print('=' * 60)
            print(f'Orçamento Total:    R$ {valor_aporte:,.2f}')
            print(f'Custo Total:        R$ {custo_total:,.2f}')
            print(f'Sobra:              R$ {sobra:,.2f}')
            print(f'Utilização:         {(custo_total/valor_aporte)*100:.1f}%')
            
            if sobra > 0:
                print(f'\nSobra de R$ {sobra:,.2f} vai para SELIC')
        return df_resultado
    else:
        return None
    
def otimizar_aporte_lp(df, valor_aporte):
    df = df.copy()

    df['Valor Ideal'] = df['% Ideal - Ref.'] * (VALOR_CARTEIRA + valor_aporte)
    df['deficit'] = (df['Valor Ideal'] - df['Total']).clip(lower=0)
    df['Qtd_nec'] = 0.0
    df['Custo_real'] = 0.0

    prob = pl.LpProblem('Otimizacao_Aporte', pl.LpMinimize)
    qt_rf, qt_rv, gap = criar_variaveis_lp(df)
    adicionar_restricoes_lp(prob, df, qt_rf, qt_rv, gap, valor_aporte)
    definir_objetivo_lp(prob, gap)
    prob.solve()
    if pl.LpStatus[prob.status] == "Optimal":
        extrair_resultados_lp(df, qt_rf, qt_rv, valor_aporte, show=True)
        return True
    else:
        return False


In [8]:
optimize = otimizar_aporte_lp(df_port, valor_aporte=VALOR_APORTE)


RESULTADO - PESQUISA OPERACIONAL


,Geo.,Ticker,Ativo,Classe,Sublasses,Cotação,Qtd_Comprada,Valor_Compra
2,BR,RBRR11,RBR Rendimento High Grade,RV,FIIs,86.890000,23.000000,1998.470000
1,BR,ALUP11,Alupar,RV,Ações,30.360000,57.000000,1730.520000
5,US,VWO,Vanguard Emerging Markets,RV,ETFs,270.906500,5.000000,1354.532500
3,US,NOBL,ProShares Dividends,RV,ETFs,561.195850,2.000000,1122.391700
4,US,SCHD,Schwab Dividend Equity,RV,ETFs,146.731575,7.000000,1027.121025
0,BR,FRFH,Fundo Renda Fixa High Grade Crédito Privado,RF,Juros Pós,1.000000,16.964775,16.964775



RESUMO FINANCEIRO:
Orçamento Total:    R$ 7,250.00
Custo Total:        R$ 7,250.00
Sobra:              R$ -0.00
Utilização:         100.0%
